In [3]:
from scipy import integrate
import numpy as np

In [97]:
def indicator(x, sub_domain_intervals):
    '''
    Define the indicator function
    indicator function is function which if x value is inside the bound, you will get 1
    Otherwise you will get 0
    '''
    '''
    Require
    x, left_bound, right_bound must have the same dimension
    '''
    '''
    Parameters
    ----------
    x : 1 x n vector representing the point to check (Time dimension should be excluded).
        It is a point that has dimension equal to the spatial dimension.

    sub_intervals : 2d (n x 2) arrays. First dimension is all the spatial dimensions, and second dimension are 
        the left and right bound of the subdomain. Note that it has the same dim
    
    return
    ------
    1 or 0, should be clear enough
    
    '''
    if len(x) != len(sub_domain_intervals[:, 0]):
        raise ValueError("Parameter dimensions do not agree.")
        
    for i in range(len(sub_domain_intervals[:, 0])):
        if x[i] < sub_domain_intervals[i, 0] or x[i] > sub_domain_intervals[i, 1]:
            return 0
    return 1

In [6]:
def compute_integral(x, t, num_x, j, endpts):
    '''
    Parameters: 
    
        f: u_j(p)(x, t)
        
        endpts: 2 x n array 
            the first column is the left endpoints of the subdomain's each of the n dimensions,
            second column is right endpoint of each of the subdomain's each of the n dimensions
            
    return:
    
        nd integral
    '''
#     This integral may need to be performed 
    return get_u_j(x, num_x, t, j) * 
    return sp.integral.nquad(get_u_j(x, num_x, t, j), endpts)

In [6]:
def get_u_j(x, num_x, t, j):
    '''
    x: data value x', a constant after stacking.
    t: time point t
    j: the feature of u that will get returned.
    '''  
#   spatial-temporal stack index
    ind = x*num_x+t
    
#   X is 2D, 
    return x[ind, j]
    

In [7]:
def get_omega_bound(index, intervals):
    ##Need to figure out how we are going to stack our intervals.
    '''
    Parameter
    --------- 
    index : index of the subdomain to get bound of
    intervals : boundary of each subdomain correspond to each dimension
        (We could make it n x m x 2 or nm x 2) n is the dimension m is the number of points
        
    return
    ------
    2d (n x 2) arrays. First dimension is all the spatial dimensions, and second dimension are left and right bound of the subdomain
    '''
    
    return intervals[index]

In [7]:
def get_theta_nonloc(j, k, kprime, intervals):
#     get how many time points are there
    num_t = np.shape(spatiotemporal_grid)[-2]
#     get how many spatial points are there
    num_x = np.prod(np.shape(spatiotemporal_grid)[:-2])
    
    theta_nonloc_p = np.zeros(num_t*num_x)
    
    for i in np.arange(theta_nonloc_p.length):
        this_t = i % num_t
        this_x = int(i/num_t)
        
#       get x(all x, this_t)
#       This currently filters the spatial temporal grid, as opposed to X
#       We mat not need this though, as we can use num_t and num_x to figure out the stacking of X.
#         n_dims  = len(np.shape(spatiotemporal_grid))
#         s = [slice(None) for i in range(n_dims)]
#         s[-2] = this_t
#         grid_t = spatiotemporal_grid[tuple(s)]
        
        coefficient = indicator(this_x, get_omega_bound(k, intervals))
        
        integral = compute_integral(this_x, this_t, num_x, j, get_omega_bound(kprime, intervals))
        
        theta_nonloc_p[i] = coefficient * integral
        
    return theta_nonloc_p

In [2]:
def full_domain_linear_weights(self, x, sub_domain_intervals):
        """
        One-dimensioal weights for integration against the 0th derivative
        of the indicator test function 1 if inside bound and 0 else. This is derived
        assuming the function to integrate is linear between grid points:
        f(x)=f_i+(x-x_i)/(x_{i+1}-x_i)*(f_{i+1}-f_i)
        so that f(x)*phi(x) are computed analytically, and the integral is
        expressed as a dot product of weights against the f_i.
        """
        phi = indicator(x, sub_domain_intervals)
        return np.array([0.5 * (x[2:] - x[1:-1]) * (phi[2:] - phi[1:-1])])

In [5]:
def full_domian_left_weights(self, x1, x2, sub_domain_intervals):
       """
        One-dimensioal weight for left-most point in integration against the 0th
        derivative of the indicator test function 1 if inside bound and 0 else. This is derived
        assuming the function to integrate is linear between grid points:
        f(x)=f_i+(x-x_i)/(x_{i+1}-x_i)*(f_{i+1}-f_i)
        so that f(x)*phi(x) are computed analytically, and the integral is
        expressed as a dot product of weights against the f_i.
        """
       phi_0 = indicator(x1, sub_domain_intervals)
       phi_1 = indicator(x2, sub_domain_intervals)
       return np.array([0.5 * (x2 - x1) * (phi_1 - phi_0)])

In [ ]:
def full_domian_right_weights(self, x1, x2, sub_domain_intervals):
       """
        One-dimensioal weight for left-most point in integration against the 0th
        derivative of the indicator test function 1 if inside bound and 0 else. This is derived
        assuming the function to integrate is linear between grid points:
        f(x)=f_i+(x-x_i)/(x_{i+1}-x_i)*(f_{i+1}-f_i)
        so that f(x)*phi(x) are computed analytically, and the integral is
        expressed as a dot product of weights against the f_i.
        """
       phi_0 = indicator(x1, sub_domain_intervals)
       phi_1 = indicator(x2, sub_domain_intervals)
       return np.array([-0.5 * (x2 - x1) * (phi_1 - phi_0)])